# Preparing the Dataset for Modeling

In [51]:
import os
import numpy as np
import pandas as pd
from google.cloud import storage

In [52]:
# Instantiates a client
storage_client = storage.Client()

In [53]:
# Define bucket name: change this to your own
bucket_name = 'optimum-vine-297907-ashay'

In [54]:
# Get GCS bucket
bucket = storage_client.get_bucket(bucket_name)

In [55]:
# Retrieve blobs in the train dataset
blobs = bucket.list_blobs(prefix='chest_xray/chest_xray/train/')

In [56]:
# Retrieve list of blobs
blob_list = []
for blob in blobs:
    blob_list.append(blob.name)

In [57]:
len(blob_list)

5219

In [58]:
print(blob_list[0])
print(blob_list[1])

chest_xray/chest_xray/train/.DS_Store
chest_xray/chest_xray/train/NORMAL/.DS_Store


In [59]:
# Remove .DS_Store from list created by the datasource system
# We don't mind doing this O(n) operation because the list size is manageable.
# For larger datasets, it may be better to simply delete this file from Google Cloud Storage.

# Append the bucket_prefix to the object file path

i, length = 0, len(blob_list)
bucket_prefix = 'gs://optimum-vine-297907-ashay/'
data = []
for blob in blob_list:
    if '.DS_Store' not in blob:
        entry = [''.join([bucket_prefix, blob]), blob.split('/')[3]]
        data.append(entry)

In [60]:
print(data[0])
print(data[1])

['gs://optimum-vine-297907-ashay/chest_xray/chest_xray/train/NORMAL/IM-0115-0001.jpeg', 'NORMAL']
['gs://optimum-vine-297907-ashay/chest_xray/chest_xray/train/NORMAL/IM-0117-0001.jpeg', 'NORMAL']


In [61]:
# convert to Pandas DataFrame
data_pd = pd.DataFrame(np.array(data))

In [62]:
data_pd.to_csv("data.csv", header=None, index=None)

In [63]:
# Upload data.csv to Google Cloud Storage
output_blob = bucket.blob('data.csv')

In [64]:
output_blob.upload_from_filename('data.csv')

# Finish